## Introduction

In this chapter, we’ll cover some features that are useful in very specific situations:

- **Unsafe Rust**: how to opt out of some of Rust’s guarantees and take responsibility for manually upholding those guarantees
- **Advanced traits**: associated types, default type parameters, fully qualified syntax, supertraits, and the newtype pattern in relation to traits
- **Advanced types**: more about the newtype pattern, type aliases, the never type, and dynamically sized types
- **Advanced functions and closures**: function pointers and returning closures
- **Macros**: ways to define code that defines more code at compile time

## Unsafe Rust

Unsafe Rust exists because, by nature, static analysis is conservative (for memory safety guarantees enforced at compile time). When the compiler tries to determine whether or not code upholds the guarantees, it’s better for it to reject some valid programs than to accept some invalid programs. Although the code might be okay, if the Rust compiler doesn’t have enough information to be confident, it will reject the code. In these cases, <span style="color:lightgreen">*you can use unsafe code to tell the compiler, “Trust me, this might look unsafe to you, but I know what I’m doing.”*</span>

### Unsafe Superpowers

To switch to unsafe Rust, use the `unsafe` keyword and then start a new block that holds the unsafe code. You can take five actions in unsafe Rust that you can’t in safe Rust, which we call unsafe superpowers. Those superpowers include the ability to:

- Dereference a raw pointer
- Call an unsafe function or method
- Access or modify a mutable static variable
- Implement an unsafe trait
- Access fields of unions

By requiring these five unsafe operations to be inside blocks annotated with unsafe, you’ll know that any errors related to memory safety must be within an `unsafe` block.

## Advanced Traits

### Specifying Placeholder Types in Trait Definitions with Associated Types

Associated types connect a type placeholder with a trait such that the trait method definitions can use these placeholder types in their signatures. E.g.

In [2]:
pub trait Iterator {
    type Item;

    fn next(&mut self) -> Option<Self::Item>;
}

## Advanced Types

### Using the Newtype Pattern for Type Safety and Abstraction

We can use the newtype pattern to abstract away some implementation details of a type: the new type can expose a public API that is different from the API of the private inner type.

Newtypes can also hide internal implementation. For example, we could provide a `People` type to wrap a `HashMap<i32, String>` that stores a person’s ID associated with their name. Code using People would only interact with the public API we provide, such as a method to add a name string to the People collection; that code wouldn’t need to know that we assign an `i32` ID to names internally.

### Creating Type Synonyms with Type Aliases

<span style="color:skyblue">*Rust provides the ability to declare a type alias to give an existing type another name. For this we use the `type` keyword.*</span>

In [3]:
type Kilometers = i32;

let x: i32 = 5;
let y: Kilometers = 5;

println!("x + y = {}", x + y);

x + y = 10


## Advanced Functions and Closures

### Function Pointers

We’ve talked about how to pass closures to functions; <span style="color:lightgreen">*you can also pass regular functions to functions!*</span> This technique is useful when you want to pass a function you’ve already defined rather than defining a new closure.

In [4]:
fn add_one(x: i32) -> i32 {
    x + 1
}

fn do_twice(f: fn(i32) -> i32, arg: i32) -> i32 {
    f(arg) + f(arg)
}

fn main() {
    let answer = do_twice(add_one, 5);

    println!("The answer is: {}", answer);
}

main()

The answer is: 12


()

### Returning Closures

Closures are represented by traits, which means you can’t return closures directly. In most cases where you might want to return a trait, you can instead use the concrete type that implements the trait as the return value of the function. However, you can’t do that with closures because they don’t have a concrete type that is returnable; you’re not allowed to use the function pointer `fn` as a return type, for example.

In [5]:
fn returns_closure() -> dyn Fn(i32) -> i32 {
    |x| x + 1
}

Error: return type cannot have an unboxed trait object

The error references the `Sized` trait again! Rust doesn’t know how much space it will need to store the closure. We saw a solution to this problem earlier. We can use a trait object:

In [9]:
fn returns_closure() -> Box<dyn Fn(i32) -> i32> {
    Box::new(|x| x + 1)
}

## Macros

We’ve used macros like `println!` throughout this book, but we haven’t fully explored what a macro is and how it works. The term `macro` refers to a family of features in Rust: declarative macros with `macro_rules!` and three kinds of procedural macros:

- Custom `#[derive]` macros that specify code added with the derive attribute used on structs and enums
- Attribute-like macros that define custom attributes usable on any item
- Function-like macros that look like function calls but operate on the tokens specified as their argument

### The Difference Between Macros and Functions

<span style="color:lightgreen">*Fundamentally, macros are a way of writing code that writes other code, which is known as metaprogramming.*</span> E.g. the `derive` attribute generates an implementation of various traits for you. We’ve also used the `println!` and `vec!` macros throughout the book. All of these macros expand to produce more code than the code you’ve written manually.

<span style="color:lightgreen">*A function signature must declare the number and type of parameters the function has. Macros, on the other hand, can take a variable number of parameters. Also, macros are expanded before the compiler interprets the meaning of the code, so a macro can, for example, implement a trait on a given type. A function can’t, because it gets called at runtime and a trait needs to be implemented at compile time. The downside to implementing a macro instead of a function is that macro definitions are more complex than function definitions because you’re writing Rust code that writes Rust code.*</span>